# Post Result Analysis

In [0]:
import numpy as np

### Discrete Mass

Calculation of the Jet Mass from a Pixelated Jet Image

### Discrete Momentum
Calculation of the Jet Momentum from a Pixelated Jet Image